In [1]:
import os

In [2]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth\\research'

In [3]:
os.chdir('/Anaconda Projects/end to end wine quality fifth/end-to-end-wine-quality-fifth')

In [4]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    # alpha: float
    # l1_ration: float
    target_column: str
    model_to_loop: str
    model_params: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
from mlProject import *
from box import ConfigBox
from sklearn.metrics import r2_score

def evaluate_models(x_train, x_test, y_train, y_test, models: ConfigBox) -> dict:
    try:
        report = {}
        logger.info(f"Entered to model evaluation list")

        for i in range(len(list(models))):
            model = eval(list(models.values())[i])
            model.fit(x_train, y_train)

            y_test_pred = model.predict(x_test)

            test_model_r2_score = r2_score(y_test, y_test_pred)

            report[list(models.keys())[i]] = test_model_r2_score

        return report

    except Exception as e:
        raise e

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = (CONFIG_FILE_PATH),
                 params_filepath = (PARAMS_FILE_PATH),
                 schema_filepath = (SCHEME_FILE_PATH),
                 model_p_filepath = (MODEL_P_FILE_PATH)):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        self.model_p = read_yaml(model_p_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        # params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        model = self.params.models
        model_p = self.model_p.params

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            # alpha=params.alpha,
            # l1_ration=params.l1_ratio,
            target_column=schema.name,
            model_to_loop = model,
            model_params = model_p
        )

        return model_trainer_config



In [9]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib
# from mlProject.utils.common import evaluate_models
from mlProject import *

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config  = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        models = self.config.model_to_loop
        model_p = self.config.model_params
        logger.info(f"Models parameters are ready to print")
        print(model_p)

        model_report:dict = evaluate_models(x_train=train_x, y_train=train_y, x_test=test_x, y_test=test_y, models=models, model=model_p)

        print(model_report)
        
        best_model_score = max(sorted(model_report.values()))

        best_model_name = list(model_report.keys())[
            list(model_report.values()).index(best_model_score)
            ]

        best_model = models[best_model_name]

        logging.info(f"Best model is {best_model_name} and its r2 score is {best_model_score}")

        # lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ration, random_state=42)

        # lr.fit(train_x, train_y)

        joblib.dump(best_model, os.path.join(self.config.root_dir, self.config.model_name))

[2024-01-23 12:32:02,565: INFO: utils: NumExpr defaulting to 4 threads.]


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-01-23 12:32:10,507: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-23 12:32:10,507: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-23 12:32:10,515: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-23 12:32:10,528: INFO: common: yaml file: model_params.yaml loaded successfully]
[2024-01-23 12:32:10,541: INFO: common: Created directory at artifacts]
[2024-01-23 12:32:10,541: INFO: common: Created directory at artifacts/model_trainer]
[2024-01-23 12:32:10,675: INFO: 876395858: Models parameters are ready to print]
{'Linear Regression': {'fit_intercept': [True, False], 'normalize': [True, False], 'copy_X': [True, False], 'n_jobs': [-1, 1, 'None']}, 'K-Neighbors Regressor': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [10, 20, 30], 'p': [1, 2], 'metric': ['minkowski', 'euclidean', 'manhattan']}, 'Decision Tree': {'criterion': ['squa